# Predicting house prices using k-nearest neighbors regression

In this notebook, you will implement k-nearest neighbors regression. You will:

Find the k-nearest neighbors of a given query input Predict the output for the query input using the k-nearest neighbors Choose the best value of k using a validation set

In [1]:
import numpy as np
import pandas as pd
import os
from IPython.display import display

# dictionary with dataset column names and their corresponding data types
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 
              'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 
              'sqft_living':float, 'floors':float, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 
              'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}

Load the train, test and validate datasets in pandas dataframe

In [2]:
# change directory to the location where the data for the notebook is located
os.chdir('D:\Anupam_Technical\Code\ML\DS_ML_Projects\Regression\data\knn')
train = pd.read_csv('kc_house_data_small_train.csv', dtype = dtype_dict)
print('Top 5 rows of training dataset')
display(train.head())
test = pd.read_csv('kc_house_data_small_test.csv', dtype = dtype_dict)
validate = pd.read_csv('kc_house_data_validation.csv', dtype = dtype_dict)

Top 5 rows of training dataset


,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3.0,1.00,1180.0,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340.0,5650.0
1,6414100192,20141209T000000,538000.0,3.0,2.25,2570.0,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690.0,7639.0
2,5631500400,20150225T000000,180000.0,2.0,1.00,770.0,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720.0,8062.0
3,2487200875,20141209T000000,604000.0,4.0,3.00,1960.0,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360.0,5000.0
4,1954400510,20150218T000000,510000.0,3.0,2.00,1680.0,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800.0,7503.0


This function extracts input and output features from a dataframe and return a tuple of input features numpy 2d array and output feature vector (numpy 1d array). Note that we need to add a column vector of all ones as the first column of the input feature matrix

In [3]:
# add a column vector of all ones to the begining of a feature matrix
def add_one_vector(X):
    one_vector = np.ones(len(X)).reshape(len(X), 1)
    return np.concatenate((one_vector, X), axis=1)


def extract_features(df, datatype_dict):
    """
    Extract input and output features from a dataframe and return a tuple of input features matrix and output 
    feature vector
    :param df: dataframe
    :param datatype_dict: dictionary with dataset column names and their corresponding data types
    :return: a tuple of input features matrix and output feature vector
    """    
    # remove the columns that are not numeric i.e. int, floats etc.
    # train.dtypes gives a pandas with index as column names and value as column data types. We filter this
    # series to remove columns of type object
    numeric_cols = pd.Series(train.dtypes).where(lambda col_dtype: col_dtype != 'object').dropna()
    feature_names = list(numeric_cols.keys().values)
    # price is the output variable
    feature_names.remove('price')
    # extract the input features from the dataframe as a numpy 2d array
    input_features = add_one_vector(df[feature_names].values)
    output_variable = df.loc[:, 'price'].values
    return input_features, output_variable

In [4]:
train_input_features, train_output = extract_features(train, dtype_dict)
cv_input_features, cv_output = extract_features(validate, dtype_dict)
test_input_features, test_output = extract_features(test, dtype_dict)

In computing distances, it is crucial to normalize features. Otherwise, for example, the ‘sqft_living’ feature (typically on the order of thousands) would exert a much larger influence on distance than the ‘bedrooms’ feature (typically on the order of ones). We divide each column of the training feature matrix by its 2-norm, so that the transformed column has unit norm.

IMPORTANT: Make sure to store the norms of the features in the training set. The features in the test and validation sets must be divided by these same norms, so that the training, test, and validation sets are normalized consistently.

In [5]:
def normalize_features(input_features):
    norm = np.sqrt(np.sum(input_features**2, axis=0))
    normalized_features = input_features / norm
    return normalized_features, norm

norm_train_input_features, train_norm = normalize_features(train_input_features)
norm_cv_input_features = cv_input_features / train_norm
norm_test_input_features = test_input_features / train_norm

# Compute a single distance

To start, let's just explore computing the “distance” between two given houses. We will take our query house to be the first house of the test set and look at the distance between this house and the 10th house of the training set.

To see the features associated with the query house, print the first row (index 0) of the test feature matrix. You should get an 18-dimensional vector whose components are between 0 and 1. Similarly, print the 10th row (index 9) of the training feature matrix.

In [6]:
print(norm_test_input_features[0])
print(norm_train_input_features[9])

[ 0.01345102  0.01551285  0.01807473  0.01759212  0.00160518  0.017059
  0.          0.05102365  0.0116321   0.01564352  0.01362084  0.02481682
  0.01350306  0.          0.01345387 -0.01346922  0.01375926  0.0016225 ]
[ 0.01345102  0.01163464  0.00602491  0.0083488   0.00050756  0.01279425
  0.          0.          0.01938684  0.01390535  0.0096309   0.
  0.01302544  0.          0.01346821 -0.01346251  0.01195898  0.00156612]


What is the Euclidean distance between the query house and the 10th house of the training set?

In [7]:
np.sqrt(np.sum((norm_train_input_features[9] - norm_test_input_features[0])**2))

0.05972359371398078

Of course, to do nearest neighbor regression, we need to compute the distance between our query house and all houses in the training set.

To visualize this nearest-neighbor search, let's first compute the distance from our query house (features_test[0]) to the first 10 houses of the training set (norm_train_input_features[0:10]) and then search for the nearest neighbor within this small set of houses. Through restricting ourselves to a small set of houses to begin with, we can visually scan the list of 10 distances to verify that our code for finding the nearest neighbor is working.

Write a loop to compute the Euclidean distance from the query house to each of the first 10 houses in the training set.

Quiz Question: Among the first 10 training houses, which house is the closest to the query house?

In [8]:
distance = {}
for i in range(10):
    distance[i] = np.sqrt(np.sum((norm_train_input_features[i] - norm_test_input_features[0])**2))
print(distance)

{0: 0.06027470916295592, 1: 0.08546881147643746, 2: 0.06149946435279315, 3: 0.05340273979294363, 4: 0.05844484060170442, 5: 0.059879215098128345, 6: 0.05463140496775461, 7: 0.055431083236146074, 8: 0.052383627840220305, 9: 0.05972359371398078}


In [9]:
distance_sorted = sorted(distance.items(), key=lambda item: item[1])
print(distance_sorted)

[(8, 0.052383627840220305), (3, 0.05340273979294363), (6, 0.05463140496775461), (7, 0.055431083236146074), (4, 0.05844484060170442), (9, 0.05972359371398078), (5, 0.059879215098128345), (0, 0.06027470916295592), (2, 0.06149946435279315), (1, 0.08546881147643746)]


# Perform one nearest neighbour regression

Now that we have the element-wise differences, it is not too hard to compute the Euclidean distances between our query house and all of the training houses. First, write a single-line expression to define a variable ‘diff’ such that ‘diff[i]’ gives the element-wise difference between the features of the query house and the i-th training house.

To test your code, print diff[-1].sum(), which should be -0.0934339605842.

In [10]:
diff = norm_train_input_features[:] - norm_test_input_features[0]
diff[-1].sum()

-0.09343399874654643

The next step in computing the Euclidean distances is to take these feature-by-feature differences in ‘diff’, square each, and take the sum over feature indices. That is, compute the sum of squared feature differences for each training house (row in ‘diff’).

By default, ‘np.sum’ sums up everything in the matrix and returns a single number. To instead sum only over a row or column, we need to specifiy the ‘axis’ parameter described in the np.sum documentation. In particular, ‘axis=1’ computes the sum across each row.

In [11]:
def compute_distance(training_examples, query_house):
    """
    Vectorized implementation of calculating the distance of a query house from each of the training examples
    :param training_examples: a matrix or numpy 2d array consisting of training data (input features)
    :param query_house: the query house
    :return: numpy 2d array whose first column is the training row index and second column is the distance from
    the query house
    """
    # subtract the query house row from each training example row
    diff_matrix = training_examples - query_house
    # now for each row in the matrix (which corresponds to each training example), calculate the sum of
    # squares of feature values ( this is done by using axis = 1 in the 2d array )
    distance = np.sqrt(np.sum(diff_matrix**2, axis=1))
    index = np.arange(0, len(distance))    
    return np.concatenate((index.reshape(-1, 1), distance.reshape(-1, 1)), axis=1)

Compute the distance of the third test example from each of the training examples. Then find out which training example is the closest to the query house. Note that the expected value is row index 382 has the min. distance of 0.0028604955575117085

In [12]:
rowindex_distance = compute_distance(norm_train_input_features[:], norm_test_input_features[2])

def get_min_distance_row_index(rowindex_distance):
    rowindex = rowindex_distance[:, 0]
    distance = rowindex_distance[:, 1]
    min_distance = np.amin(distance)
    min_distance_index = np.where(distance == min_distance)
    return rowindex[min_distance_index], min_distance

min_row_index, min_distance = get_min_distance_row_index(rowindex_distance)
print(int(min_row_index[0]), min_distance)

382 0.0028604955575117085


What is the predicted value of the query house (third test example) based on 1-nearest neighbor regression?

In [13]:
# Since the query house ( i.e. the third test example) is closest to the 382nd training example, hence the predicted 
# value of the query house will be same as the price of the 382nd training example
train_output[382]

249000.0

# Perform k-nearest neighbor regression

Using the functions above, implement a function that takes in

the value of k; the feature matrix for the instances; and the feature of the query and returns the indices of the k closest training houses. For instance, with 2-nearest neighbor, a return value of [5, 10] would indicate that the 6th and 11th training houses are closest to the query house.

In [14]:
def k_nearest_neighbours(k, training_examples, query_house):
    rownumber_distance = compute_distance(training_examples, query_house)
    # sort the 2d array on the index column in ascending order
    # You can call .argsort() on the column you want to sort, and it will give you an array of row indices 
    # that sort that particular column which you can pass as an index to your original array.
    rownumber_distance_sorted = rownumber_distance[rownumber_distance[:, 1].argsort()]
    return rownumber_distance_sorted[0:k, :]

Take the query house to be third house of the test set (norm_test_input_features[2]). What are the indices of the 4 training houses closest to the query house?

In [15]:
knn_rowindex_distance = k_nearest_neighbours(4, norm_train_input_features[:], norm_test_input_features[2])
print(knn_rowindex_distance[:, 0].astype(int))

[ 382 1149 4087 3142]


Now that we know how to find the k-nearest neighbors, write a function that predicts the value of a given query house. For simplicity, take the average of the prices of the k nearest neighbors in the training set. The function should have the following parameters:

<ul>
    <li>the value of k;</li>
    <li>the feature matrix for the instances;</li>
    <li>the output values (prices) of the instances; and</li>
    <li>the feature of the query, whose price we’re predicting.</li>
</ul>
The function should return a predicted value of the query house.

In [16]:
def predict_house_price_custom(k, train_input_features, train_output, query_house):    
    k_rowindex_distance = k_nearest_neighbours(k, train_input_features, query_house)
    # get the rowindex of the k nearest neighbours and get their corresponding prices
    k_rowindex = k_rowindex_distance[:, 0].astype(int)
    # get the mean of the k nearest house prices, this is the predicted price of the query house
    return np.mean(train_output[k_rowindex])

Make predictions for the first 10 houses in the test set, using k=10. What is the index of the house in this query set that has the lowest predicted value? What is the predicted value of this house?

In [17]:
query_house_predicted_price = []

for query_house_index in range(10):
    query_house = norm_test_input_features[query_house_index]
    predicted_price = predict_house_price_custom(10, norm_train_input_features, train_output, query_house)
    query_house_predicted_price.append((query_house_index, predicted_price))

# sort on the basis of predicted prices in ascending order    
sorted_query_house_predicted_price = sorted(query_house_predicted_price, key=lambda item:item[1])
print(sorted_query_house_predicted_price)
print('\nHouse index in test set of 10 houses with lowest predicted price: {}'
      .format(sorted_query_house_predicted_price[0][0]))

[(6, 350032.0), (3, 430200.0), (1, 431860.0), (9, 457235.0), (2, 460595.0), (8, 484000.0), (7, 512800.7), (5, 667420.0), (4, 766750.0), (0, 881300.0)]

House index in test set of 10 houses with lowest predicted price: 6


# Choosing the best value of k using a validation set

There remains a question of choosing the value of k to use in making predictions. Here, we use a validation set to choose this value. Write a loop that does the following:

For k in [1, 2, … 15]:

Make predictions for the VALIDATION data using the k-nearest neighbors from the TRAINING data. Compute the RSS on VALIDATION data Report which k produced the lowest RSS on validation data.

In [34]:
from sklearn.metrics import r2_score

# Predicts multiple house prices using a knn implementation
# predict_house_price_single: Function that predicts a single query house price based on a knn implementation
# input_features: input features matrix for houses whose price is to be predicted
# target_variable: the actual price of the houses in input features matrix
# Returns: A numpy 2d array with first column contain house index, second column the predicted price and
# third column the actual price
def predict_prices_multiple(k, predict_price_single, input_features, target_variable):
    queryhouseindex_predictedprice_actualprice = []
    for query_house_index in range(len(input_features)):
        query_house = input_features[query_house_index]
        actual_price = target_variable[query_house_index]
        predicted_price = predict_price_single(k, norm_train_input_features, train_output, query_house)
        queryhouseindex_predictedprice_actualprice.append([query_house_index, predicted_price, actual_price])
    queryhouseindex_predictedprice_actualprice = np.array(queryhouseindex_predictedprice_actualprice)        
    return queryhouseindex_predictedprice_actualprice

def get_optimized_kvalue(list_kvalues, predict_house_price):
    k_rss = []
    k_r2score = []
    for k in list_kvalues:    
        cv_actual_predicted_prices = predict_prices_multiple(k, predict_house_price, norm_cv_input_features, cv_output)
        # now calculate the residual sum of squares ( (predicted value - actual value)**2 ) over the entire validation set
        price_diff = (cv_actual_predicted_prices[:, 1] - cv_actual_predicted_prices[:, 2])**2
        r2score = r2_score(cv_actual_predicted_prices[:, 2], cv_actual_predicted_prices[:, 1])
        rss = np.sum(price_diff)
        k_rss.append((k, rss))
        k_r2score.append((k, r2score))
        print('k: {} --> rss: {}, r2score: {:.4f}'.format(k, rss, r2score))        
    sorted_k_rss = sorted(k_rss, key=lambda item:item[1])    
    sorted_k_r2score = sorted(k_r2score, key=lambda item:item[1], reverse=True)    
    return sorted_k_r2score[0][0]


In [33]:
optimized_k = get_optimized_kvalue(np.arange(15)+1, predict_house_price_custom)
print('\n The value of k with best r2 score on validation data is: {}'.format(optimized_k))

k: 1 --> rss: 105453830251561.0, r2score: 0.4258
k: 2 --> rss: 83445073504025.5, r2score: 0.5456
k: 3 --> rss: 72692096019202.56, r2score: 0.6042
k: 4 --> rss: 71946721652091.69, r2score: 0.6083
k: 5 --> rss: 69846517419718.6, r2score: 0.6197
k: 6 --> rss: 68899544353180.836, r2score: 0.6248
k: 7 --> rss: 68341973450051.09, r2score: 0.6279
k: 8 --> rss: 67361678735491.5, r2score: 0.6332
k: 9 --> rss: 68372727958976.09, r2score: 0.6277
k: 10 --> rss: 69335048668556.74, r2score: 0.6225
k: 11 --> rss: 69523855215598.83, r2score: 0.6214
k: 12 --> rss: 69049969587246.17, r2score: 0.6240
k: 13 --> rss: 70011254508263.69, r2score: 0.6188
k: 14 --> rss: 70908698869034.34, r2score: 0.6139
k: 15 --> rss: 71106928385945.16, r2score: 0.6128

 The value of k with best r2 score on validation data is: 8


# K Nearest Neighbours with scikit-learn

In [35]:
from sklearn.neighbors import KNeighborsRegressor

def predict_house_price_scikit(k, train_input_features, train_output, query_house):
    knn_regressor = KNeighborsRegressor(n_neighbors = k, weights='distance')
    knn_regressor.fit(train_input_features, train_output)    
    return knn_regressor.predict(query_house.reshape(1, -1))

optimized_k_scikit = get_optimized_kvalue(np.arange(15)+1, predict_house_price_scikit)
print('\nThe value of k (using scikit nearest neighbors) with best r2 score on validation data is: {}'
      .format(optimized_k_scikit))

k: 1 --> rss: [1.05451198e+14], r2score: 0.4258
k: 2 --> rss: [8.3482811e+13], r2score: 0.5454
k: 3 --> rss: [7.26620105e+13], r2score: 0.6044
k: 4 --> rss: [7.16405256e+13], r2score: 0.6099
k: 5 --> rss: [6.95642264e+13], r2score: 0.6212
k: 6 --> rss: [6.8640568e+13], r2score: 0.6263
k: 7 --> rss: [6.7961639e+13], r2score: 0.6300
k: 8 --> rss: [6.69584338e+13], r2score: 0.6354
k: 9 --> rss: [6.76112457e+13], r2score: 0.6319
k: 10 --> rss: [6.79035784e+13], r2score: 0.6303
k: 11 --> rss: [6.79588549e+13], r2score: 0.6300
k: 12 --> rss: [6.77070525e+13], r2score: 0.6313
k: 13 --> rss: [6.84091455e+13], r2score: 0.6275
k: 14 --> rss: [6.90942193e+13], r2score: 0.6238
k: 15 --> rss: [6.9224199e+13], r2score: 0.6231

The value of k (using scikit nearest neighbors) with best r2 score on validation data is: 8


For the optimized k value predict the house prices on test set and compute the accuracy of predictions both in terms of percentage of exact match plus the mean squared error and the r2 scores both for the scikit and custom knn models

In [47]:
predicted_test1 = predict_house_price_custom(8, norm_train_input_features, train_output, norm_train_input_features[1])
actual_test1 = train_output[1]
print('actal: {:.2f}, predicted: {:.2f}'.format(actual_test1, predicted_test1))

actal: 538000.00, predicted: 622500.00


In [38]:
test_predictions = predict_prices_multiple(8, predict_house_price_scikit, norm_test_input_features, test_output)
print('Completed predictions')
test_predicted_price = test_predictions[:, 1]
test_actual_price = test_predictions[:, 2]
prediction_error = test_predicted_price - test_actual_price
for row in test_predictions:
    print(row)
test_r2score = r2_score(test_predicted_price, test_actual_price)
print('\nThe r2 score with k=8 on test set predictions is: {:.4f}'.format(test_r2score))

Completed predictions
[0 array([878939.27542628]) 650000.0]
[1 array([455781.57397594]) 485000.0]
[2 array([392360.02197029]) 438000.0]
[3 array([493695.26768916]) 535000.0]
[4 array([779022.14645364]) 785000.0]
[5 array([663965.52743391]) 975000.0]
[6 array([340847.76689232]) 287000.0]
[7 array([513333.4970537]) 355000.0]
[8 array([476574.37695417]) 305000.0]
[9 array([479849.50618052]) 518500.0]
[10 array([458853.09889674]) 535000.0]
[11 array([419360.63562354]) 665000.0]
[12 array([439390.984727]) 585188.0]
[13 array([284419.34677388]) 181000.0]
[14 array([744870.33668022]) 455000.0]
[15 array([519350.12112739]) 592500.0]
[16 array([667784.09456071]) 693000.0]
[17 array([282608.28035497]) 425000.0]
[18 array([558852.96618684]) 405000.0]
[19 array([492435.62565257]) 315000.0]
[20 array([460948.88994464]) 487000.0]
[21 array([962485.03084944]) 880000.0]
[22 array([807895.75314868]) 559900.0]
[23 array([760498.4397559]) 679900.0]
[24 array([751119.20569185]) 380000.0]
[25 array([406007

[1208 array([777691.61013727]) 984000.0]
[1209 array([655711.37420073]) 485000.0]
[1210 array([635575.90248515]) 685000.0]
[1211 array([415080.95185272]) 194000.0]
[1212 array([389794.32279728]) 267000.0]
[1213 array([519715.18405465]) 525000.0]
[1214 array([485701.93891577]) 729000.0]
[1215 array([1098428.93826179]) 935000.0]
[1216 array([465368.89314746]) 499000.0]
[1217 array([507812.15413297]) 570000.0]
[1218 array([553558.60944755]) 352500.0]
[1219 array([262031.93093738]) 199000.0]
[1220 array([418102.78799989]) 236000.0]
[1221 array([713987.54182157]) 989900.0]
[1222 array([993658.43060117]) 967500.0]
[1223 array([489538.50623866]) 490000.0]
[1224 array([322788.91844263]) 160000.0]
[1225 array([319749.6644436]) 239000.0]
[1226 array([257262.26404504]) 179000.0]
[1227 array([206108.92667956]) 190000.0]
[1228 array([1439770.72338973]) 465000.0]
[1229 array([963808.11322567]) 840000.0]
[1230 array([382528.97459637]) 375000.0]
[1231 array([646203.88153676]) 160000.0]
[1232 array([73

A negative R2 score implies that the predictions made by the knn model are worse off than using the mean price of all test house samples ( basically a constant fit to the data ) as the prediction value.